# Twitter data collection


In [1]:
import pickle
#import os
#from collections import Counter
#import json
#import datetime as dt
import twitter
import sqlite3
import pandas as pd
import numpy as np

In [2]:
Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

In [3]:
#!pip install twitter

## Authorizing an application to access Twitter account data

In [4]:
auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth, retry=True)

## Collecting results in a sqlite database

In [5]:
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

# Create base
#------------------------------------------------------------
# Table: statuses
#------------------------------------------------------------
c.execute('''CREATE TABLE statuses(
        id                    Int NOT NULL ,
        coordinates           Varchar (250) ,
        created_at            Varchar (250) ,
        favorite_count        Int ,
        lang                  Varchar (25) ,
        place                 Varchar (2500) ,
        scopes                Varchar (2500) ,
        retweet_count         Int ,
        source                Varchar (2500) ,
        txt                   Varchar (250) ,
        withheld_in_countries Varchar (250) ,
        retweeted_status_id   Int ,
        in_reply_to_status_id Int ,
        in_reply_to_user_id   Int,
        quoted_status_id      Int ,
        user_id               Int ,
        PRIMARY KEY (id )
        );''')

#------------------------------------------------------------
# Table: users
#------------------------------------------------------------
c.execute('''CREATE TABLE users(
        id                    Int NOT NULL ,
        screen_name           Varchar (250) ,
        created_at            Varchar (100) ,
        description           Varchar (2500) ,
        favourites_count      Int ,
        followers_count       Int ,
        friends_count         Int ,
        lang                  Varchar (25) ,
        listed_count          Int ,
        location              Varchar (250) ,
        name                  Varchar (25) ,
        statuses_count        Int ,
        PRIMARY KEY (id )
        );''')

#------------------------------------------------------------
# Table: mentions
#------------------------------------------------------------

c.execute('''CREATE TABLE mentions(
        status_id Int NOT NULL ,
        user_id Int NOT NULL ,
        PRIMARY KEY (status_id ,user_id )
);''')

#------------------------------------------------------------
# Table: tags
#------------------------------------------------------------

c.execute('''CREATE TABLE tags(
        status_id Int NOT NULL ,
        htg    Varchar (140) NOT NULL ,
        PRIMARY KEY (status_id ,htg )
);''')

The use of informations from the table statuses and users that are suceptible to change with the time - such as 'retweet_count' or 'friend_count' - are subject to caution here as their values caption is ponctual and not actualised. The registered value is only their value when their where capted for the first time.

In [6]:
def add_user(u,c):
    #Create the user entity and main attributes
    c.execute('''INSERT INTO users(id,screen_name,created_at,description,favourites_count,
            followers_count,friends_count,lang,listed_count,
            location,name,statuses_count)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?) ''', 
          (str(u['id']),u['screen_name'],u['created_at'],u['description'],str(u['favourites_count']),
            str(u['followers_count']),str(u['friends_count']),u['lang'],str(u['listed_count']),
              u['location'],u['name'],str(u['statuses_count'])))

In [7]:
def add_status(s,c):
    #Create the status entity and main attributes
    c.execute('''
    INSERT INTO 
        statuses(id,created_at,favorite_count,
        lang,retweet_count,source,txt)
    VALUES (?, ?,? ,?  ,? ,? ,?)''', (str(s['id']),s['created_at'],str(s['favorite_count']),
        s['lang'],
        str(s['retweet_count']),
        s['source'], 
        s['text']))
    
    #Facultative attributes (None if absent)
    for k in ['in_reply_to_status_id','place','coordinates','in_reply_to_user_id']:
        if s[k] is not None:
            v=(k, str(s[k]), str(s['id']))
            c.execute('''
            UPDATE statuses
            SET %s = ?
            WHERE id = ?''' % k, (str(s[k]), str(s['id'])))
    
    #Facultative attributes (not in s if absent)
    
    for k in ['scopes','withheld_in_countries','quoted_status_id']:
        if k in s :
            c.execute('''
            UPDATE statuses
            SET %s=?
            WHERE id=?'''% k,(str(s[k]),str(s['id'])))

    #Other entities
    for k in ['retweeted_status','user']:
        if k in s :
            c.execute('''
                UPDATE statuses
                SET %s_id=?
                WHERE id=?''' % k, (str(s[k]['id']),str(s['id'])))

            
            
    ###########################################USERS##############################################
    if (c.execute("SELECT * FROM users WHERE id=?", (str(s['user']['id']),)).fetchone())==None:
        u=twitter_api.users.show(id=str(s['user']['id']))
        add_user(u,c)
    #else MAJ user

    if not s['in_reply_to_user_id']==None:
        if (c.execute("SELECT * FROM users WHERE id=?", (str(s['in_reply_to_user_id']),)).fetchone())==None:
            u=twitter_api.users.show(id=str(s['in_reply_to_user_id']))
            add_user(u,c)
        #else MAJ user
        
    for u in s['entities']['user_mentions']:
        if (c.execute('''SELECT * FROM mentions 
                    WHERE status_id = ? AND user_id = ?''', (str(s['id']),str(u['id']))).fetchone())==None:
            #We do not register the multiple mentions of a user in a tweet
            c.execute('''INSERT INTO mentions(status_id ,user_id)
                        VALUES (?, ?)''', (str(s['id']),str(u['id'])))

            if (c.execute('''SELECT * FROM users WHERE id=?''', (str(u['id']),)).fetchone())==None:
                u=twitter_api.users.show(id=str(u['id']))
                add_user(u,c)       

    ################################# HASHTAGS #################################################
    for h in s['entities']['hashtags']:
        if (c.execute('''SELECT * FROM tags 
                    WHERE status_id=? AND htg = ?''',(str(s['id']),h['text'].lower())).fetchone())==None:
            #Ajouter le htg 
            c.execute('''INSERT INTO tags(status_id, htg) VALUES (?,?)''',(str(s['id']),h['text'].lower()))

In [8]:
htgs = ['#egalim','#egalimentation','#EtatsGenerauxdelAlimentation','#EGA']
w = ["états généraux de l'alimentation"]
nb = 100
lg='fr'

In [9]:
###Collection of data
req=c.execute('''SELECT id FROM statuses ORDER by id DESC''').fetchone()

if req == None:
    sid=0
else : sid=req[0]
    
statuses=[]
for q in htgs :
    l_nw_st=100
    s=[]
    mid=10**20

    while l_nw_st==100:
        search_results = twitter_api.search.tweets(q=q, count=nb,  result_type='recent',  since_id=sid, max_id=mid)
        nw_st=search_results['statuses']
        l_nw_st=len(nw_st)
        s.extend(nw_st)
        if len(s)>0 : mid=s[-1]['id']-1   
    statuses.extend(s)


In [10]:
l_st=len(statuses)
print(l_st)

19


In [11]:
[('{:-^100}'.format(s["user"]["screen_name"])+ '-'*5 + str(s['retweet_count'])+ '-'*2,s["text"],'') 
 for s in statuses][:5]

[('---------------------------------------------clems76LH---------------------------------------------------3--',
  "RT @Prefet14: #egalim .@StTRAVERT @Min_Agriculture est à #Caen pour l'ouverture des États Généraux de l'#Alimentation normands\n#Normandie #…",
  ''),
 ('------------------------------------------AgroSupMediadoc------------------------------------------------0--',
  "#EGalim : les ateliers en région | Alim'agri | @scoopit https://t.co/C8X161JbQG",
  ''),
 ('----------------------------------------------ConfAura---------------------------------------------------2--',
  "RT @ConfPaysanne: Venez voter pour les propositions de la Conf' aux #EGAlim : en qques jours nos propositions sont parmi les + votées\n http…",
  ''),
 ('----------------------------------------------Prefet14---------------------------------------------------0--',
  '#egalim États généraux de l’#alimentation\n- Création et partage de la valeur \n- Alimentation saine, sûre, durable e… https://t.co/q8ENV6g8V

In [12]:
for s in statuses:
    ls=len(c.execute('''SELECT * FROM statuses''').fetchall())
    lu=len(c.execute('''SELECT * FROM users''').fetchall())
    lh=len(c.execute('''SELECT * FROM tags''').fetchall())
    lm=len(c.execute('''SELECT * FROM mentions''').fetchall())
    print (ls,'  ',lu,'  ',lh,'  ',lm)
    #########################################STATUSES#############################################
    #main status
    if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['id']),)).fetchone())==None:
        add_status(s,c)
    
    #replied status
    if s['in_reply_to_status_id'] is not None:
        if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['in_reply_to_status_id']),)).fetchone())==None:
            r=twitter_api.statuses.show(id=s['in_reply_to_status_id'])
            add_status(r,c)        
                                           
    #quoted status
    if 'quoted_status_id' in s:
        if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['quoted_status_id']),)).fetchone())==None:
            q=twitter_api.statuses.show(id=s['quoted_status_id'])
            add_status(q,c)

     

7657    3680    13741    9868
7658    3681    13745    9871
7659    3681    13746    9872
7660    3681    13747    9873
7661    3681    13749    9873
7662    3681    13750    9873
7663    3683    13754    9875
7664    3683    13755    9875
7665    3683    13759    9876
7666    3683    13760    9876
7667    3684    13761    9876
7668    3684    13765    9879
7669    3685    13766    9881
7670    3685    13770    9884
7671    3685    13773    9886
7672    3685    13775    9889
7673    3686    13776    9890
7674    3686    13777    9894
7675    3686    13778    9894


In [13]:
ls=len(c.execute('''SELECT * FROM statuses''').fetchall())
lu=len(c.execute('''SELECT * FROM users''').fetchall())
lh=len(c.execute('''SELECT * FROM tags''').fetchall())
lm=len(c.execute('''SELECT * FROM mentions''').fetchall())
print (ls,'  ',lu,'  ',lh,'  ',lm)

7676    3686    13779    9896


In [14]:
conn.commit()

In [15]:
rows=c.execute('''SELECT users.name, statuses.txt, statuses.created_at
                    FROM users, statuses
                    WHERE 
                    statuses.user_id==users.id
                    AND users.screen_name = ?''',('carmen_etche',))

In [16]:
for r in rows.fetchall():
    print('{:-^90}'.format(r[0]),r[2],r[1])

------------------------------------Carmen Etcheverry------------------------------------- Wed Sep 20 09:08:12 +0000 2017 RT @Agathe_CIWF: Avec @CIWF_FR aux #EGAlim pour defendre l'etiquetage des modes de production pour repondre aux attentes des consommateurs!…
------------------------------------Carmen Etcheverry------------------------------------- Wed Sep 20 09:18:00 +0000 2017 Prix justes, contractualisation et alimentation de qualité : ensemble, ça peut fonctionner ! #EGalim https://t.co/VguDKuiDEr
------------------------------------Carmen Etcheverry------------------------------------- Fri Sep 22 11:16:49 +0000 2017 RT @FNEasso: #EGAlim : découvrez les demandes de 50 organisations de la société civile par ici &gt;  https://t.co/Yx8osH3SXS https://t.co/oK5p…
------------------------------------Carmen Etcheverry------------------------------------- Wed Aug 02 06:54:53 +0000 2017 .@N_Hulot "Je vais me battre pour que les aides au maintien AB soient à l'ordre du jour des #EGalim" C

## Data exploration

In [17]:
rows=c.execute('''SELECT users.screen_name, statuses.txt 
                    FROM users, statuses
                    WHERE statuses.user_id==users.id''').fetchall()

In [18]:
len(rows)

7675

In [19]:
l=[[name,status] 
     for (name,status) in rows]

In [20]:
l[:5]

[['COSTESLionelEr',
  'RT @M_Orphelin: Atelier "Comment mieux rémunérer les agriculteurs?" des #EGAlim. Nous travaillons ce matin sur de nouveaux types de contrat…'],
 ['Edelaloyere',
  'RT @alain_cardinaux: #EGalim 66% des prises de parole chez les agriculteurs proviennent de la @fnsea, tous médias confondus (selon @agence_…'],
 ['ChLambert_FNSEA',
  "Pour partager l'objectif de créer&amp;mieux répartir la valeur dans la chaine alim, échange avec les cadres… https://t.co/YCUYbSeIwo"],
 ['ClementineBabo1',
  '"Pourquoi la spirale #déflationniste doit nous interroger ? On ne peut plus avoir tout moins #cher..." https://t.co/osbDFlpGly'],
 ['amo_thinktwice',
  "[#EGAlim] Les cinq propositions de l'Agence Bio pour que la France maintienne son autosuffisance en produits bio… https://t.co/LI7OcVosxC"]]

In [21]:
rows=c.execute('''SELECT htg, count(*) 
                    FROM tags
                    GROUP BY htg
                    ORDER BY count(*) DESC''').fetchall()

In [22]:
l=[[htg,count] 
     for (htg,count) in rows]
i=[r[0]for r in rows]
d=[r[1]for r in rows]

In [23]:
tags_ser=pd.Series(d,i)
tags_ser[:10]

egalim            6453
ega                873
agriculture        317
soutenonslabio     310
afecoan            234
bio                200
alimentation       198
directan           195
ceta               119
agriculteurs       111
dtype: int64

In [24]:
tags_ser

egalim                   6453
ega                       873
agriculture               317
soutenonslabio            310
afecoan                   234
bio                       200
alimentation              198
directan                  195
ceta                      119
agriculteurs              111
rtm                       111
space2017                 108
bigard                    106
chambagri                 102
egabzh                     89
comeco                     83
coop_agri17                80
inra                       74
élevage                    70
leclerc                    66
agricole                   65
lait                       54
fipronil                   51
competitivité              48
mayenne                    44
rennes                     44
lrem                       42
votéquitable               42
glyphosate                 41
beelite                    38
                         ... 
transitionenergetique       1
travailequitable            1
truth     

In [25]:
rows=c.execute('''SELECT users.screen_name, count(*) 
                    FROM users, statuses
                    WHERE statuses.user_id==users.id
                    GROUP BY users.id
                    ORDER BY count(*) DESC''').fetchall()

In [26]:
l=[[name,count] 
     for (name,count) in rows]
i=[r[0]for r in rows]
d=[r[1]for r in rows]

In [27]:
users_ser=pd.Series(d,i)
users_ser[:15]

moreaujb23         58
CoopdeFrance       55
lacoopagricole     47
LeTweetPol         47
UnionAgricole87    39
ChouetteAgile      38
FrankGuehennec     37
philippe_jehan     37
GrainesInsoumis    37
Inra_France        36
ChambagriBzh       34
Young_Spray        32
alain_cardinaux    31
olivier_frey       31
calmette_boris     31
dtype: int64

### Coocurence of tags in statuses

In [28]:
rows=c.execute('''SELECT statuses.id, tags.htg
                    FROM statuses, tags
                    WHERE statuses.retweeted_status_id is NULL
                    AND statuses.id = tags.status_id''').fetchall()

In [30]:
# Building of a dictionnary of statuses and tags
d={}
for r in rows :
    if r[0] in d :
        #If the status already in the dictionary, affect 1 to the value correstponding to the tag
        d[r[0]][r[1]]=1
    else : 
        #Else, create a dictionnary for the status and affect 1 to the tag
        d[r[0]]={r[1]:1}
        
d
        
    

{909755154689413121: {'cher': 1, 'déflationniste': 1},
 909792961239617538: {'egalim': 1},
 909788831536308224: {'couts': 1, 'egalim': 1, 'offre': 1},
 909783353762689027: {'agriculture': 1,
  'alimentation': 1,
  'consommation': 1,
  'egalim': 1,
  'élevage': 1},
 909775977303232512: {'egalim': 1},
 909774510697705473: {'agriculteurs': 1, 'egalim': 1},
 909753652218417152: {'bio': 1, 'egalim': 1},
 909752897700864000: {'agriculture': 1, 'alimentation': 1, 'egalim': 1},
 909750416224768000: {'agroalimentaire': 1, 'egalim': 1},
 909685316457508865: {'cice': 1, 'exclulsa': 1},
 909721991866970112: {'egalim': 1, 'prixdulait': 1},
 909720671726563328: {'egalim': 1},
 909720093164220416: {'egalim': 1},
 909718557113688065: {'egalim': 1},
 909714936082915328: {'egalim': 1},
 909714022290862080: {'egalim': 1, 'lrem': 1},
 909711733048135681: {'egalim': 1},
 909710025240797184: {'egalim': 1},
 909709191081807873: {'egalim': 1},
 909708634690670592: {'space2017': 1},
 909708315059458048: {'egal

In [31]:
list(d.values())

[{'cher': 1, 'déflationniste': 1},
 {'egalim': 1},
 {'couts': 1, 'egalim': 1, 'offre': 1},
 {'agriculture': 1,
  'alimentation': 1,
  'consommation': 1,
  'egalim': 1,
  'élevage': 1},
 {'egalim': 1},
 {'agriculteurs': 1, 'egalim': 1},
 {'bio': 1, 'egalim': 1},
 {'agriculture': 1, 'alimentation': 1, 'egalim': 1},
 {'agroalimentaire': 1, 'egalim': 1},
 {'cice': 1, 'exclulsa': 1},
 {'egalim': 1, 'prixdulait': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1, 'lrem': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'space2017': 1},
 {'egalim': 1, 'pme': 1},
 {'alimentation': 1,
  'bio': 1,
  'durable': 1,
  'egalim': 1,
  'foodtech': 1,
  'santé': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1, 'fiscalité': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'egalim': 1},
 {'agriculture': 1, 'egalim': 1},
 {'egalim': 1},
 {'agriculteurs': 1, 'consommateurs': 1, 'egalim': 1},
 {'coopérative': 1},
 {'egalim': 1},
 {'bio': 1, 'egalim

In [34]:
df=pd.DataFrame(list(d.values()),d.keys())
df

,43einhalb,8bitphotolab,8bits,abattage,abattoirs,abio2017,abonnés,accaparementdeterre,accorddeparis,acofal,...,émission,énergie,équitable,états,étiquetage,กระทรวงดิจิทัล,ภาษีไปไหน,สรอ,แอพพลิเคชั่น,恐山
909755154689413121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909792961239617538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909788831536308224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909783353762689027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909775977303232512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909774510697705473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909753652218417152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909752897700864000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909750416224768000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
909685316457508865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df[df.isnull()]=0

In [60]:
df

,43einhalb,8bitphotolab,8bits,abattage,abattoirs,abio2017,abonnés,accaparementdeterre,accorddeparis,acofal,...,émission,énergie,équitable,états,étiquetage,กระทรวงดิจิทัล,ภาษีไปไหน,สรอ,แอพพลิเคชั่น,恐山
909755154689413121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909792961239617538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909788831536308224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909783353762689027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909775977303232512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909774510697705473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909753652218417152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909752897700864000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909750416224768000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
909685316457508865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
#Co-occurence matrix
coocc=df.T.dot(df)
coocc

,43einhalb,8bitphotolab,8bits,abattage,abattoirs,abio2017,abonnés,accaparementdeterre,accorddeparis,acofal,...,émission,énergie,équitable,états,étiquetage,กระทรวงดิจิทัล,ภาษีไปไหน,สรอ,แอพพลิเคชั่น,恐山
43einhalb,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8bitphotolab,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8bits,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abattage,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
abattoirs,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abio2017,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abonnés,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accaparementdeterre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accorddeparis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acofal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Creer une matrice de coocurence des htg
- Creer une matrice de lien entre les comptes (qui site qui, qui retweet qui...)
- Creer une matrice de lien entre comptes et htg (=> grpe de htg et grpe de comptes)
    --> Quels utilisateurs utilisent les même htg
    --> Quels htg sont utilisés par les mêmes utilisateurs
- analyse du texte des tweet?